## 名字と部署を避けてグループ分けするためのコード

In [1]:
import math
import random
import re
import pandas as pd

In [2]:
# 人のデータを格納するリスト

In [3]:
df = pd.read_excel("ad.xlsx", encoding="cp932")

In [4]:
def evenly_distribute_people(total_people, group_num):
    """
    Divide total_people into group_num groups as evenly as possible
    """
    people_per_group = math.floor(total_people / group_num)
    remainder = total_people % group_num
    group_sizes = {}
    for i in range(1, group_num + 1):
        group_sizes[str(i)] = people_per_group
    for i in range(remainder):
        group_sizes[str(i+1)] += 1
    return group_sizes

In [5]:
group_dict = evenly_distribute_people(len(df), 2)

In [6]:
def convert_group(group):
    match = re.match(r"(\d+)コース希望", str(group))
    if match:
        group_dict[str(match.group(1))] -= 1
        return str(match.group(1))
    
    else:
        return "希望なし"

df["希望"] = df["希望"].apply(convert_group)

In [7]:
df["希望"]

0     希望なし
1     希望なし
2     希望なし
3     希望なし
4     希望なし
5     希望なし
6     希望なし
7     希望なし
8     希望なし
9     希望なし
10    希望なし
11    希望なし
12       1
13    希望なし
14    希望なし
15       1
16    希望なし
17    希望なし
18    希望なし
19    希望なし
Name: 希望, dtype: object

In [8]:
df['名字'] = df['氏名'].str.replace('　', ' ').str.split(' ', expand=True)[0]

In [9]:
import pandas as pd
from random import shuffle

def group_request_assign(df, group_num):
    # Step 1: Create a new column 'group' and initialize it with 'None'
    df['group'] = '個別判断'
    # Step 2: Sort the dataframe by '希望' column and assign groups to people with preferences first
#     df = df.sort_values(by='希望')
    for i in range(len(df)):
        if df.iloc[i]['希望'] != "希望なし":
            df.at[df.index[i], 'group'] = df.iloc[i]['希望']
    df = df.sample(frac=1)
    # Step 4: Assign groups to people without preferences, while avoiding same surname and department
    group_list = [str(i+1) for i in range(group_num)]
    print(group_list)
    
    for i in range(len(df)):
        if df.iloc[i]['group'] == '個別判断':
            for j in group_list:
                if group_dict[j] == 0:
                    continue
                same_surname = df[(df['group'] == j) & (df['名字'] == df.iloc[i]['名字'])]
                same_department = df[(df['group'] == j) & (df['本来所属 部'] == df.iloc[i]['本来所属 部'])]
                if len(same_surname) == 0 and len(same_department) == 0:
                    df.at[df.index[i], 'group'] = j
                    group_dict[j] -= 1
                    break
    return df

In [10]:
group_request_assign(df, 2)

['1', '2']


,従業員コード,氏名,AD研修コースID,AD研修コース,AD研修クラスID,AD研修クラス,担当コースID,担当コース,担当クラスID,担当クラス,...,ロケーションコード,ロケーション,２年目AD,講師歴（AD歴、総括AD歴）,講師歴（その他）,社内経歴役割,NOｿｰﾄ用,希望,名字,group
4,3188783,水野 竜治,101,1コース,101,1クラス,NaN,NaN,NaN,NaN,...,160,元町製造技術,NaN,2022年度 ５０特別研修 ２２年度_５０特別研修_車生技 アドバイザー／2018年度 新任...,2018年度 ６０特別研修 １８年度＿６０特別研修＿車生技 TCSトレーナー／2016年度 ...,2016年 ＣＸ会 専門部員／2015年 ＣＸ会 支部長／2014年 ＣＸ会 副支部長／20...,14405,希望なし,水野,1
11,3192656,谷本 和徳,101,1コース,101,1クラス,NaN,NaN,NaN,NaN,...,109,高岡工場,NaN,NaN,NaN,2019年 ＣＸ会 支部長／2018年 ＣＸ会 副支部長／2016年 トヨタ労組 職場委員長...,25004,希望なし,谷本,1
17,3191974,岩崎 博則,101,1コース,101,1クラス,NaN,NaN,NaN,NaN,...,115,田原工場,NaN,2022年度 ５０特別研修 ２２年度_５０特別研修_田原 アドバイザー／2018年度 ６０特...,2017年度 ９０研修 １７年度＿９０研修＿全社 TPSトレーナー／2015年度 ６０特別研...,2019年 ＣＸ会 副支部長／2016年 トヨタ労組 職場委員長／2015年 トヨタ労組 職...,31565,希望なし,岩崎,1
16,3185196,水上 広樹,101,1コース,101,1クラス,NaN,NaN,NaN,NaN,...,114,衣浦工場,NaN,2022年度 ５０特別研修 ２２年度_５０特別研修_衣浦 アドバイザー／2016年度 ６０特...,NaN,2018年 ＣＸ会 支部長／2003年 ＥＸ会 支部長／2002年 ＥＸ会 副支部長／199...,30659,希望なし,水上,1
10,3183153,辻岡 清,101,1コース,101,1クラス,NaN,NaN,NaN,NaN,...,108,上郷工場,NaN,2017年度 新任組長研修 １７年度＿新任組長研修＿全社 アドバイザー,NaN,2022年 巧会 幹事／2019年 ＣＸ会 支部長／2018年 ＣＸ会 副支部長／2015年...,23869,希望なし,辻岡,1
7,3195863,川越 一成,101,1コース,101,1クラス,NaN,NaN,NaN,NaN,...,103,本社工場,NaN,NaN,2019年度 ７０特別研修 １９年度＿７０特別研修＿本工 TPSトレーナー／2018年度 新...,2019年 ＣＸ会 専門部員／2015年 トヨタ労組 局長／2014年 トヨタ労組 局長／2...,20378,希望なし,川越,1
1,3191349,芦野 仁,101,1コース,101,1クラス,NaN,NaN,NaN,NaN,...,102,本社技術,NaN,2019年度 ６０特別研修 １９年度＿６０特別研修＿技術 アドバイザー／2012年度 ７０特...,2019年度 新任ＳＸ級研修 １９年度＿新任ＳＸ級研修＿技術 TCSトレーナー／2018年度...,2018年 トヨタ労組 職場委員長／2017年 トヨタ労組 職場委員長／2010年 ＳＸ会 ...,5249,希望なし,芦野,1
5,3173899,佐々木 英雄,101,1コース,101,1クラス,NaN,NaN,NaN,NaN,...,118,東富士研究所,NaN,2020年度 ５０特別研修 ２０年度＿５０特別研修_パワモノ アドバイザー,NaN,2015年 ＣＸ会 副支部長／1994年 トヨタ労組 職場委員／1993年 トヨタ労組 職場...,18166,希望なし,佐々木,1
18,3194346,松野 浩一,101,1コース,101,1クラス,NaN,NaN,NaN,NaN,...,115,田原工場,NaN,2011年度 技能専修コース １１年度＿技能専修コース＿全社 総括アドバイザー／2010年度...,NaN,2021年 ＣＸ会 副支部長／2018年 トヨタ労組 職場委員長／2017年 トヨタ労組 職...,32247,希望なし,松野,2
15,3184003,樋渡 邦弘,101,1コース,101,1クラス,NaN,NaN,NaN,NaN,...,114,衣浦工場,NaN,2014年度 ６０特別研修 １４年度＿６０特別研修＿全社 アドバイザー,2019年度 ９０研修 １９年度＿９０研修＿衣浦 TPSトレーナー／2018年度 ７０特別研...,2020年 ＣＸ会 専門部員／2018年 ＣＸ会 副支部長／2010年 ＨＵＲＥＡＩ活動 部...,30463,1,樋渡,1


In [11]:

    # Step 3: Shuffle the dataframe to randomize the order of people without preferences
    df = df.sample(frac=1)
    # Step 4: Assign groups to people without preferences, while avoiding same surname and department
    group_list = [i+1 for i in range(group_num)]
    print(group_list)
    for i in range(len(df)):
        print(i)
        if df.iloc[i]['group'] == '個別判断':
#         if pd.isna(df.at[i,'group']):
            for j in group_list:
                print(j)
                same_surname = df[(df['group'] == j) & (df['名字'] == df.iloc[i]['名字'])]
                same_department = df[(df['group'] == j) & (df['本来所属 部'] == df.iloc[i]['本来所属 部'])]
                if len(same_surname) == 0 and len(same_department) == 0:
                    df.at[df.index[i], 'group'] = j
                    break
    return df

NameError: name 'group_num' is not defined

In [ ]:
group_assign(df, 2)

In [ ]:
df.columns

In [ ]:
df["group"]

In [ ]:
for i in range(len(df)):
    print(pd.isna(df.at[i,'group']))

In [ ]:
group_assign(df, 2)